In [16]:
from tensorflow.keras.models import Model
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input,LSTM,Dense
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pickle
#initialize all variables 
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

In [17]:
#read dataset file
with open('eng-french.txt','r',encoding='utf-8') as f:
    rows=f.read().split('\n')

#read first 10,000 rows from dataset     
for row in rows[:10000]:

    #split input and target by '\t'=tab
    input_text,target_text = row.split('\t')

    #add '\t' at start and '\n' at end of text.
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text.lower())
    target_texts.append(target_text.lower())

    #split character from text and add in respective sets
    input_characters.update(list(input_text.lower())) # list ne word ko letter by leeter brake kiya and then store in set
    target_characters.update(list(target_text.lower()))

print(input_characters)
print(target_characters)
# print(input_texts)
# print(target_texts)


{'8', '6', '.', 'y', ':', "'", 'd', 'f', 'i', '9', 'm', ',', 'c', 'n', 'p', 'r', '-', 'z', 'a', 'b', '7', '!', '5', '4', 'k', 'l', '&', '$', 'w', 'u', 't', '0', ' ', '?', '’', 'x', 'g', '3', '2', '1', 'e', 'o', 'v', 'q', 's', 'h', 'j'}
{'é', '8', 'ë', '6', '.', 'y', '\u2009', 'î', "'", 'd', ':', 'û', '\u202f', 'f', '\t', 'i', '9', 'm', ',', 'c', 'n', ')', 'p', 'r', '-', 'è', 'z', 'a', 'b', '!', '5', '‘', 'ô', '»', 'k', 'l', '&', 'ï', '$', '«', 'w', 'u', 't', '0', ' ', '?', '’', 'x', 'g', '3', 'œ', '1', 'ê', '\xa0', 'ù', 'e', '(', 'o', 'v', '\n', 'q', 'â', 'ç', 'à', 's', 'h', 'j'}


In [4]:
#sort input and target characters 
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
print('Input Characters  ',input_characters)
print('Target Characters  ',target_characters)
#get the total length of input and target characters
num_en_chars = len(input_characters)
num_dec_chars = len(target_characters)

#get the maximum length of input and target text.
max_input_length = max([len(i) for i in input_texts])  # list copmrehension data read and then will pick the highest value in the list
max_target_length = max([len(i) for i in target_texts])

print("number of encoder characters : ",num_en_chars)
print("number of decoder characters : ",num_dec_chars)

print("maximum input length : ",max_input_length)
print("maximum target length : ",max_target_length)

Input Characters   [' ', '!', '$', '&', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '’']
Target Characters   ['\t', '\n', ' ', '!', '$', '&', "'", '(', ')', ',', '-', '.', '0', '1', '3', '5', '6', '8', '9', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '«', '»', 'à', 'â', 'ç', 'è', 'é', 'ê', 'ë', 'î', 'ï', 'ô', 'ù', 'û', 'œ', '\u2009', '‘', '’', '\u202f']
number of encoder characters :  47
number of decoder characters :  67
maximum input length :  16
maximum target length :  59


In [5]:
def bagofcharacters(input_texts,target_texts):

    #initialize encoder , decoder input and target data.
    en_in_data=[] ; dec_in_data=[] ; dec_tr_data=[]

    #padding variable with first character as 1 as rest all 0.
    pad_en=[1]+[0]*(len(input_characters)-1)
    pad_dec=[0]*(len(target_characters)) ; pad_dec[2]=1

    #countvectorizer for one hot encoding as we want to tokenize character so
    #analyzer is true and None the stopwords action.
    cv = CountVectorizer(binary=True,tokenizer=lambda txt:
    txt.split(),stop_words=None,analyzer='char')
  
    for i,(input_t,target_t) in enumerate(zip(input_texts,target_texts)):
        #fit the input characters into the CountVectorizer functionbx 
        # Training the encoder input with all the unique chars possible  
        cv_inp= cv.fit(input_characters)
    
        #transform the input text from the help of CountVectorizer fit.
        #it character present than put 1 and 0 otherwise.
        en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())
        cv_tar= cv.fit(target_characters)    
        dec_in_data.append(cv_tar.transform(list(target_t)).toarray().tolist())

        #decoder target will be one timestep ahead because it will not consider 
        #the first character i.e. '\t'.
        dec_tr_data.append(cv_tar.transform(list(target_t)[1:]).toarray().tolist())
        #add padding variable if the length of the input or target text is smaller
      #than their respective maximum input or target length. 
        if len(input_t) < max_input_length:
            for _ in range(max_input_length-len(input_t)):
                en_in_data[i].append(pad_en)
        if len(target_t) < max_target_length:
            for _ in range(max_target_length-len(target_t)):
                dec_in_data[i].append(pad_dec)
        if (len(target_t)-1) < max_target_length:
            for _ in range(max_target_length-len(target_t)+1):
                dec_tr_data[i].append(pad_dec)

    #convert list to numpy array with data type float32
    en_in_data=np.array(en_in_data,dtype="float32")
    dec_in_data=np.array(dec_in_data,dtype="float32")
    dec_tr_data=np.array(dec_tr_data,dtype="float32")
    return en_in_data,dec_in_data,dec_tr_data


In [6]:
#create input object of total number of encoder characters
en_inputs = Input(shape=(None, num_en_chars))

#create LSTM with the hidden dimension of 256
#return state=True as we don't want output sequence.
encoder = LSTM(256, return_state=True)

#discard encoder output and store hidden and cell state.
en_outputs, state_h, state_c = encoder(en_inputs)
en_states = [state_h, state_c]

In [7]:
#create input object of total number of decoder characters
dec_inputs = Input(shape=(None, num_dec_chars))

#create LSTM with the hidden dimension of 256
#return state and return sequences as we want output sequence.
dec_lstm = LSTM(256, return_sequences=True, return_state=True)

#initialize the decoder model with the states on encoder.
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=en_states)

#Output layer with shape of total number of decoder characters 
# softmaz is used to see for the given value neuron will activate or not 
dec_dense = Dense(num_dec_chars, activation="softmax")
dec_outputs = dec_dense(dec_outputs)

In [8]:
#create Model and store all variables 
model = Model([en_inputs, dec_inputs], dec_outputs)

pickle.dump({'input_characters':input_characters,'target_characters':target_characters, 'max_input_length':max_input_length, 'max_target_length':max_target_length, 'num_en_chars':num_en_chars, 'num_dec_chars':num_dec_chars}, open("training_data.pkl", "wb"))

#load the data and train the model
en_in_data,dec_in_data,dec_tr_data = bagofcharacters(input_texts,target_texts)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

model.fit(
    [en_in_data, dec_in_data],
    dec_tr_data,
    batch_size=64,
    epochs=200,
    validation_split=0.2,
)

# Save model
model.save("s2s")

C:\Users\gsmul.000\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\gsmul.000\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\feature_extraction\text.py:551: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


Epoch 1/200
125/125 [==============================] - 17s 119ms/step - loss: 1.2712 - accuracy: 0.7170 - val_loss: 1.1234 - val_accuracy: 0.6795
Epoch 2/200
125/125 [==============================] - 14s 113ms/step - loss: 0.8936 - accuracy: 0.7524 - val_loss: 0.9629 - val_accuracy: 0.7407
Epoch 3/200
125/125 [==============================] - 14s 112ms/step - loss: 0.7449 - accuracy: 0.7955 - val_loss: 0.8196 - val_accuracy: 0.7660
Epoch 4/200
125/125 [==============================] - 14s 112ms/step - loss: 0.6508 - accuracy: 0.8132 - val_loss: 0.7467 - val_accuracy: 0.7807
Epoch 5/200
125/125 [==============================] - 14s 112ms/step - loss: 0.5961 - accuracy: 0.8266 - val_loss: 0.7005 - val_accuracy: 0.7940
Epoch 6/200
125/125 [==============================] - 14s 114ms/step - loss: 0.5590 - accuracy: 0.8366 - val_loss: 0.6631 - val_accuracy: 0.8058
Epoch 7/200
125/125 [==============================] - 14s 115ms/step - loss: 0.5313 - accuracy: 0.8444 - val_loss: 0.6401 -

INFO:tensorflow:Assets written to: s2s\assets


INFO:tensorflow:Assets written to: s2s\assets


In [9]:
#summary and model plot
model.summary()

plot_model(model, to_file='Model_plot.png', show_shapes=True, show_layer_names=True)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 47)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None, 67)]   0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        311296      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                              

In [10]:
datafile = pickle.load(open("training_data.pkl","rb"))
input_characters = datafile['input_characters']
target_characters = datafile['target_characters']
max_input_length = datafile['max_input_length']
max_target_length = datafile['max_target_length']
num_en_chars = datafile['num_en_chars']
num_dec_chars = datafile['num_dec_chars']

cv=CountVectorizer(binary=True,tokenizer=lambda txt:
txt.split(),stop_words=None,analyzer='char') 

In [11]:

#Inference model
#load the model
model = models.load_model("s2s")

#construct encoder model from the output of second layer
#discard the encoder output and store only states.
enc_outputs, state_h_enc, state_c_enc = model.layers[2].output 

#add input object and state from the layer.
en_model = Model(model.input[0], [state_h_enc, state_c_enc])

#create Input object for hidden and cell state for decoder
#shape of layer with hidden or latent dimension
dec_state_input_h = Input(shape=(256,), name="input_3")
dec_state_input_c = Input(shape=(256,), name="input_4")
dec_states_inputs = [dec_state_input_h, dec_state_input_c]

#add input from the encoder output and initialize with states.
dec_lstm = model.layers[3]
dec_outputs, state_h_dec, state_c_dec = dec_lstm(
    model.input[1], initial_state=dec_states_inputs
)

dec_states = [state_h_dec, state_c_dec]
dec_dense = model.layers[4]
dec_outputs = dec_dense(dec_outputs)

#create Model with the input of decoder state input and encoder input
#and decoder output with the decoder states.
dec_model = Model(
    [model.input[1]] + dec_states_inputs, [dec_outputs] + dec_states
)

In [12]:
def bagofcharacters_test(input_t):
    cv=CountVectorizer(binary=True,tokenizer=lambda txt:
    txt.split(),stop_words=None,analyzer='char') 
    en_in_data=[] ; pad_en=[1]+[0]*(len(input_characters)-1)

    cv_inp= cv.fit(input_characters)
    en_in_data.append(cv_inp.transform(list(input_t)).toarray().tolist())

    if len(input_t)< max_input_length:
      for _ in range(max_input_length-len(input_t)):
        en_in_data[0].append(pad_en)

    return np.array(en_in_data,dtype="float32")

In [13]:
def decode_sequence(input_seq):
    #create a dictionary with a key as index and value as characters.
    reverse_target_char_index = dict(enumerate(target_characters))
    #get the states from the user input sequence
    states_value = en_model.predict(input_seq)

    #fit target characters and 
    #initialize every first character to be 1 which is '\t'.
    #Generate empty target sequence of length 1.
    co=cv.fit(target_characters) 
    target_seq=np.array([co.transform(list("\t")).toarray().tolist()],dtype="float32")

    #if the iteration reaches the end of text than it will be stop the it
    stop_condition = False
    #append every predicted character in decoded sentence
    decoded_sentence = ""

    while not stop_condition:
        #get predicted output and discard hidden and cell state.
        output_chars, h, c = dec_model.predict([target_seq] + states_value)

        #get the index and from the dictionary get the character.
        char_index = np.argmax(output_chars[0, -1, :])
        text_char = reverse_target_char_index[char_index]
        decoded_sentence += text_char
        
        # Exit condition: either hit max length
        # or find a stop character.
        if text_char == "\n" or len(decoded_sentence) > max_target_length:
            stop_condition = True
        #update target sequence to the current character index.
        target_seq = np.zeros((1, 1, num_dec_chars))
        target_seq[0, 0, char_index] = 1.0
        states_value = [h, c]
    #return the decoded sentence
    return decoded_sentence

In [14]:
english = "How are you?"
en_in_data = bagofcharacters_test(english)
french = decode_sequence(en_in_data)
print(english, ' --> ', french)

C:\Users\gsmul.000\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\gsmul.000\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\feature_extraction\text.py:551: UserWarning: The parameter 'tokenizer' will not be used since 'analyzer' != 'word'
  warnings.warn(


How are you?  -->  comment il va ?

